In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q torch
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q pyarrow==14.0.1
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('HuggingFace')

In [ ]:
from huggingface_hub import whoami

user = whoami(token=hf_token)['name']

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random

from peft import PeftConfig, PeftModel, PeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [ ]:
from typing import TypeVar, List, Tuple, Dict, NoReturn
from datasets import DatasetDict, Dataset, IterableDatasetDict, IterableDataset

T = TypeVar("T")

DataPointType=DatasetDict | Dataset | IterableDatasetDict | IterableDataset
DataPointTrain =Dataset | List | T
DataPointTest =Dataset | List | T

from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast
from transformers.models.mistral.modeling_mistral import MistralForCausalLM

In [ ]:
# BASE_MODEL_NAME = "mistralai/Mistral-7B-v0.1"
BASE_MODEL_NAME="mistralai/Mistral-7B-Instruct-v0.2"

BLUEPRINT_ADAPTER_CHECKPOINTS_NAME = "ckpts_mistral_7B_blueprint_planning_v2"
DETAIL_ADAPTER_CHECKPOINTS_NAME = "ckpts_mistral_7B_detailed_planning_v2"

MERGED_MODEL_NAME="Categorical_Planning_Mistral-7B-Instruct-v0.2"

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from transformers import AutoTokenizer

def load_tokenizer(model_id: str = BASE_MODEL_NAME)->LlamaTokenizerFast:
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

    # Check if the pad token is already in the tokenizer
    if '[<pad>]' not in tokenizer.get_vocab():
        # Add the pad token
        tokenizer.add_tokens(['[<pad>]'])
        print("Added pad token")

    # Set the pad token
    tokenizer.pad_token = '[<pad>]'

    tokenizer.padding_side='left'

    # Print the pad token ids
    print('Tokenizer pad token ID:', tokenizer.pad_token_id)
    return tokenizer

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenizer=load_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Added pad token
Tokenizer pad token ID: 32000


In [ ]:
print("EOS token:", tokenizer.eos_token)
print("EOS token id:", tokenizer.eos_token_id)

print("Pad token: ", tokenizer.pad_token)
print("Pad token ID: ", tokenizer.pad_token_id)

print(tokenizer)

EOS token: </s>
EOS token id: 2
Pad token:  [<pad>]
Pad token ID:  32000
LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[<pad>]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[<pad>]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}


In [ ]:
from datasets import load_dataset

def custom_dataset_load(data_dir: str)->DataPointType:
    data = load_dataset(data_dir, split="train")
    return data

In [ ]:
def format_dataset_blueprint_fn(data_points: DataPointType) -> DataPointType:
    template = "Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task."

    INST = "### Instruction:"
    RESP = "### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{template}\n\n",
                    f"{INST}{x['Instruction'].strip()}\n\n",
                    f"{RESP}{x['Blueprint'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['Blueprint'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_blueprint_instruct_fn(data_points: DataPointType) -> DataPointType:
    template = "Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task."

    B_INST, E_INST = "[INST]", "[/INST]"
    RESP = "### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{B_INST}{template.strip()}{x['Instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}{x['Blueprint']}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['Blueprint']}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_detail_fn(data_points: DataPointType) -> DataPointType:
    template = "Below is an instruction that describes a composite task. Provide the structured detail planning for the following task."

    INST = "### Instruction:"
    RESP = "### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{template}\n\n",
                    f"{INST}{x['Instruction'].strip()}\n\n",
                    f"{RESP}{x['Detailed'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['Detailed'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def format_dataset_detail_instruct_fn(data_points: DataPointType) -> DataPointType:
    template = "Below is an instruction that describes a composite task. Provide the structured detail planning for the following task."

    B_INST, E_INST = "[INST]", "[/INST]"
    RESP = "### Response:"

    # Create the formatted text
    formatted_dataset = data_points.map(
        lambda x: {
            "prompt": "".join(
                [
                    f"{B_INST}{template.strip()}{x['Instruction'].strip()}{E_INST}\n\n",
                    f"{RESP}{x['Detailed'].strip()}",
                ]
            ),
            "response": "".join(
                [
                    f"{RESP}{x['Detailed'].strip()}",
                ]
            ),
        }
    )
    return formatted_dataset

In [ ]:
def shuffle_tokenize_batch(data:DataPointType, tokenizer:LlamaTokenizerFast)->DataPointType:
    data = data.shuffle(seed=1234)  # Shuffle dataset here
    data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
    return data

In [ ]:
def split_train_test_dataset(data:DataPointType, split_size: float = 0.1)->Tuple[DataPointTrain,DataPointTest]:
    data = data.train_test_split(test_size=split_size)
    train_data = data["train"]
    test_data = data["test"]
    return train_data, test_data

In [ ]:
def format_prediction_blueprint_fn(data_point: DataPointType) -> str:
    template="Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. "

    INST = "### Instruction:"
    RESP = "### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
        [
            f"{template}\n\n",
            f"{INST}{data_point['Instruction'].strip()}\n\n",
            f"{RESP}",
        ]
    )
    return formatted_prediction

In [ ]:
def format_prediction_blueprint_instruct_fn(data_point: DataPointType) -> str:
    template="Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. "

    B_INST, E_INST = "[INST]", "[/INST]"
    RESP = "### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
        [
            f"{B_INST}{template}{data_point['Instruction'].strip()}{E_INST}\n\n",
            f"{RESP}",
        ]
    )
    return formatted_prediction

In [ ]:
def format_prediction_detail_fn(data_point: DataPointType) -> str:
    template="Below is an instruction that describes a composite task. Provide the structured detailed planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. "

    INST = "### Instruction:"
    RESP = "### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
        [
            f"{template}\n\n",
            f"{INST}{data_point['Instruction'].strip()}\n\n",
            f"{RESP}",
        ]
    )
    return formatted_prediction

In [ ]:
def format_prediction_detail_instruct_fn(data_point: DataPointType) -> str:
    template="Below is an instruction that describes a composite task. Provide the structured detailed planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. "

    B_INST, E_INST = "[INST]", "[/INST]"
    RESP = "### Response:"

    # Create the formatted text
    formatted_prediction = "".join(
        [
            f"{B_INST}{template}{data_point['Instruction'].strip()}{E_INST}\n\n",
            f"{RESP}",
        ]
    )
    return formatted_prediction

In [ ]:
import torch
from transformers import StoppingCriteria

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.keyword_ids = []
        self.max_keyword_len = 0
        for keyword in keywords:
            cur_keyword_ids = tokenizer(keyword).input_ids
            if len(cur_keyword_ids) > 1 and cur_keyword_ids[0] == tokenizer.bos_token_id:
                cur_keyword_ids = cur_keyword_ids[1:]
            if len(cur_keyword_ids) > self.max_keyword_len:
                self.max_keyword_len = len(cur_keyword_ids)
            self.keyword_ids.append(torch.tensor(cur_keyword_ids))
        self.tokenizer = tokenizer
        self.start_len = input_ids.shape[1]

    def call_for_batch(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        offset = min(output_ids.shape[1] - self.start_len, self.max_keyword_len)
        self.keyword_ids = [keyword_id.to(output_ids.device) for keyword_id in self.keyword_ids]
        for keyword_id in self.keyword_ids:
            if (output_ids[0, -keyword_id.shape[0]:] == keyword_id).all():
                return True
        outputs = self.tokenizer.batch_decode(output_ids[:, -offset:], skip_special_tokens=True)[0]
        for keyword in self.keywords:
            if keyword in outputs:
                return True
        return False

    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        outputs = []
        for i in range(output_ids.shape[0]):
            outputs.append(self.call_for_batch(output_ids[i].unsqueeze(0), scores))
        return all(outputs)

In [ ]:
def get_completion(prompt:str, model:MistralForCausalLM, tokenizer:LlamaTokenizerFast, keywords:List[str], keyword_stopping_criteria:KeywordsStoppingCriteria)->str:
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    stopping_criteria = keyword_stopping_criteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
      output_ids = model.generate(
          inputs=input_ids,
          use_cache=True,
          max_new_tokens=512,
          stopping_criteria=[stopping_criteria],
          attention_mask=None,
          do_sample=False,
          output_attentions=True,
          output_scores=True,
          return_dict_in_generate=True,
      )

    output = tokenizer.decode(output_ids['sequences'][0, input_ids.shape[1]:],skip_spectial_tokens=True).strip()

    return output

In [ ]:
def generate_handle_permutations(
    handle_opts: List[str], weights: List[float]
) -> List[List[str]]:
    # Pair each failure handling option with its weight and sort by the weights in descending order
    paired_sorted_opts = sorted(list(zip(handle_opts, weights)), key=lambda x: -x[1])

    # Initialize the result list to store the permutations
    result = []

    # Define a recursive helper function to perform backtracking
    def backtrack(curr_perm: List[str]):
        # If the current permutation is complete, add it to the result list
        if len(curr_perm) == len(handle_opts):
            result.append(curr_perm.copy())
            return

        for option, _ in paired_sorted_opts:
            if option not in curr_perm:
                # Choose the current option to add to the permutation
                curr_perm.append(option)
                # Explore further by continuing to build the permutation
                backtrack(curr_perm)
                # Unchoose the last option to try a different option in the next iteration
                curr_perm.pop()

    # Start the recursive backtracking from an empty permutation
    backtrack([])

    return result

In [ ]:
data=custom_dataset_load('/content/drive/MyDrive/data')

Resolving data files:   0%|          | 0/202 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data_blueprint=format_dataset_blueprint_instruct_fn(data)

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [ ]:
data_detail=format_dataset_detail_instruct_fn(data)

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [ ]:
processed_data_blueprint=shuffle_tokenize_batch(data_blueprint,tokenizer)

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [ ]:
processed_data_detail=shuffle_tokenize_batch(data_detail,tokenizer)

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [ ]:
processed_df_blueprint = processed_data_blueprint.to_pandas()
processed_df_blueprint.head(5)

,Blueprint,Instruction,Detailed,prompt,response,input_ids,attention_mask
0,{'Translate short story into screenplay': {'ne...,"Translate a short story into a screenplay, the...",{'Convert the narrative and dialogues of a bri...,[INST]Below is an instruction that describes a...,### Response:{'Translate short story into scre...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,{'Select artworks': {'next_modes': ['Write bio...,"Create a virtual art gallery exhibit, featurin...","{""Curate a collection of digital artworks that...",[INST]Below is an instruction that describes a...,### Response:{'Select artworks': {'next_modes'...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,{'Analyze sentiment': {'next_modes': ['Visuali...,Analyze the sentiment of tweets related to a s...,{'Utilize advanced text analysis tools to gaug...,[INST]Below is an instruction that describes a...,### Response:{'Analyze sentiment': {'next_mode...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,{'Capture drone footage': {'next_modes': ['Ana...,Analyze traffic patterns from drone footage of...,{'Collect high-resolution aerial footage of ta...,[INST]Below is an instruction that describes a...,### Response:{'Capture drone footage': {'next_...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,{'Topic selection': {'next_modes': ['Script wr...,Develop a software tutorial series including s...,{'Identify fundamental programming concepts an...,[INST]Below is an instruction that describes a...,### Response:{'Topic selection': {'next_modes'...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
processed_df_detail = processed_data_detail.to_pandas()
processed_df_detail.head(5)

,Blueprint,Instruction,Detailed,prompt,response,input_ids,attention_mask
0,{'Translate short story into screenplay': {'ne...,"Translate a short story into a screenplay, the...",{'Convert the narrative and dialogues of a bri...,[INST]Below is an instruction that describes a...,### Response:{'Convert the narrative and dialo...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,{'Select artworks': {'next_modes': ['Write bio...,"Create a virtual art gallery exhibit, featurin...","{""Curate a collection of digital artworks that...",[INST]Below is an instruction that describes a...,"### Response:{""Curate a collection of digital ...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,{'Analyze sentiment': {'next_modes': ['Visuali...,Analyze the sentiment of tweets related to a s...,{'Utilize advanced text analysis tools to gaug...,[INST]Below is an instruction that describes a...,### Response:{'Utilize advanced text analysis ...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,{'Capture drone footage': {'next_modes': ['Ana...,Analyze traffic patterns from drone footage of...,{'Collect high-resolution aerial footage of ta...,[INST]Below is an instruction that describes a...,### Response:{'Collect high-resolution aerial ...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,{'Topic selection': {'next_modes': ['Script wr...,Develop a software tutorial series including s...,{'Identify fundamental programming concepts an...,[INST]Below is an instruction that describes a...,### Response:{'Identify fundamental programmin...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
_,test_data_blueprint=split_train_test_dataset(processed_data_blueprint)

In [ ]:
_,test_data_detail=split_train_test_dataset(processed_data_detail)

In [ ]:
import torch
from transformers import AutoModelForCausalLM,BitsAndBytesConfig

def load_quantized_model(model_id: str = BASE_MODEL_NAME,device_map:Dict[str,int]={"": 0})->MistralForCausalLM:

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map=device_map
    )
    return model

In [ ]:
model=load_quantized_model()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Update pad token id in model and its config
model.pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Check if they are equal
assert model.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID"

In [ ]:
model.pad_token_id=tokenizer.pad_token_id
model.config.pad_token_id=tokenizer.pad_token_id

model.config.use_cache = True

In [ ]:
stop_keywords = ["</s>"," </s>", "</s> "]

In [ ]:
test_data_0=test_data_blueprint[0]

In [ ]:
print(test_data_0["Blueprint"])

{'Translate article': {'next_modes': ['Create infographic'], 'failure': ['Retry', 'Switch to Different Mode', 'Request Human Intervention', 'Log and Analyze']}, 'Create infographic': {'next_modes': ['Analyzer'], 'failure': ['Retry', 'Fallback Content', 'Quality Assurance Check', 'Partial Content Delivery']}, 'Analyzer': {'next_modes': [], 'failure': None}}


In [ ]:
print(test_data_0["Detailed"])

{'Carefully convert the article into the target language, ensuring technical terms are accurately translated': {'next_modes': ["Design an infographic that visually represents the article's key points, using relevant data and quotations for emphasis"], 'failure': ['Employ a more experienced translator for a second opinion', 'Opt for a simpler version of the article if translation complexities arise', 'Request feedback from a native speaker in the scientific community', 'Record and review translation challenges for future reference']}, "Design an infographic that visually represents the article's key points, using relevant data and quotations for emphasis, and ensuring the design is accessible and engaging": {'next_modes': ["Critically review the infographic for accuracy, visual impact, and alignment with the translated article's messaging"], 'failure': ['Redesign the infographic with feedback from peers', 'Utilize pre-designed templates if design quality is lacking', 'Verify all data po

In [ ]:
pred_prompt_blueprint=format_prediction_blueprint_instruct_fn(test_data_0)
print(pred_prompt_blueprint)

[INST]Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. Translate a popular science article into another language and then create an infographic that summarizes it.[/INST]

### Response:


In [ ]:
test_data_90 = test_data_blueprint[90]
pred_prompt_blueprint_b=format_prediction_blueprint_instruct_fn(test_data_90)
print(pred_prompt_blueprint_b)

[INST]Below is an instruction that describes a composite task. Provide the structured blueprint planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. Write and illustrate a children's book, then create an audiobook version.[/INST]

### Response:


In [ ]:
pred_prompt_detail=format_prediction_detail_instruct_fn(test_data_0)
print(pred_prompt_detail)

[INST]Below is an instruction that describes a composite task. Provide the structured detailed planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. Translate a popular science article into another language and then create an infographic that summarizes it.[/INST]

### Response:


In [ ]:
pred_prompt_detail_b=format_prediction_detail_instruct_fn(test_data_90)
print(pred_prompt_detail_b)

[INST]Below is an instruction that describes a composite task. Provide the structured detailed planning for the following task. Your response should be in the form of a dictionary with transitions (one-to-one or one-to-many depending on the task) between different subtasks and appropriate failure handling options. Write and illustrate a children's book, then create an audiobook version.[/INST]

### Response:


In [ ]:
additional_info="""Use the following structure as your response:
{'subtask-i': {'next_modes': ['...,subtask-j,...,subtask-n'],
'failure': [a list of strings including different failure handling options]},...,
'subtask-n': {'next_modes': ['Analyzer'], 'failure': [a list of strings including different failure handling options]},
'Analyzer': {'next_modes': [], 'failure': None}}. """

In [ ]:
additional_info_blueprint=additional_info+"""Use ONLY SHORT PHRASES for each subtask and corresponding failure options. Make 'Analyzer' as the final subtask with corresponding failure options as empty list."""

In [ ]:
additional_info_blueprint

"Use the following structure as your response:\n{'subtask-i': {'next_modes': ['...,subtask-j,...,subtask-n'],\n'failure': [a list of strings including different failure handling options]},...,\n'subtask-n': {'next_modes': ['Analyzer'], 'failure': [a list of strings including different failure handling options]},\n'Analyzer': {'next_modes': [], 'failure': None}}. Use ONLY SHORT PHRASES for each subtask and corresponding failure options. Make 'Analyzer' as the final subtask with corresponding failure options as empty list."

In [ ]:
additional_info_detail="""Use the following structure as your response:
{'detailed description of subtask-i': {'next_modes': ['..., detailed description of subtask-j,...,detailed description of subtask-n'],
'detailed description of failure': [a list of strings including different failure handling options with detailed description]},...,
'detailed description of subtask-n': {'next_modes': ['detailed description of Analyzer'], 'failure': [a list of strings including different failure handling options with detailed description]},
'detailed description of Analyzer': {'next_modes': [], 'failure': None}}. """

In [ ]:
# with model.disable_adapter(): Use this if you intend to disable adapters (i.e., after merging the adapters with the base model)
output_base_blueprint=get_completion(pred_prompt_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_blueprint)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
MistralModel is using MistralSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


[<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>]Four tasks and Transitions for Translating the Given Instructions for Translating the Popular Science Article T

In [ ]:
output_base_blueprint_with=get_completion(pred_prompt_blueprint+additional_info_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_blueprint_with)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[<pad>][<pad>][<pad>] Translation:
{'Translation:
{'article_source': 'English',
'target_language': 'Spanish',
'article_text': '',
'next_modes': ['Infographic_Creation'],
'failure': ['Unable to find article text', 'Unable to access internet for translation']},
Infographic_Creation:
{'article_text': '',
'next_modes': ['Design_Infographic', 'Failure_Handler'],
'failure': ['Unable to understand article text', 'Unable to access design tools']},
Design_Infographic:
{'article_text': '',
'next_modes': ['Finalize_Infographic'],
'failure': ['Unable to design infographic']},
Finalize_Infographic:
{'infographic': '',
'next_modes': ['Analyzer'],
'failure': ['Unable to save infographic']},
Analyzer:
{'infographic': '',
'next_modes': [],
'failure': None}}

This blueprint describes a composite task where a popular science article is translated into another language and then an infographic is created summarizing it. The 'Translation' subtask is responsible for fetching the article text, translating it 

In [ ]:
with model.disable_adapter():
  output_base_blueprint_with_b=get_completion(pred_prompt_blueprint_b+additional_info_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
  print(output_base_blueprint_with_b)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
{'Write_story': {'next_modes': ['Illustrate_story'], 'failure': ['Story_unclear']},
 'Illustrate_story': {'next_modes': ['Record_voice'], 'failure': ['Illustrations_incomplete']},
 'Record_voice': {'next_modes': ['Edit_voice'], 'failure': ['Recording_unclear']},
 'Edit_voice': {'next_modes': ['Create_audiobook'], 'failure': ['Editing_unsatisfactory']},
 'Create_audiobook': {'next_modes': ['Analyzer'], 'failure': ['Audiobook_creation_[<pad>]Failed']},
 'Analyzer': {'next_modes': [], 'failure': []}}
```

### Explanation:

1. Write_story: Write the story for the children's book.
2. Illustrate_story: Illustrate the story with appropriate pictures.
3. Record_voice: Record the voice for the audiobook version.
4. Edit_voice: Edit the recorded voice for clarity and quality.
5. Create_audiobook: Create the final audiobook version.
6. Analyzer: Analyze the completion of the entire project.

Failure handling options:
- Story_unclear: If the story is unclear, go back to Write_story and r

In [ ]:
# with model.disable_adapter():
output_base_detail=get_completion(pred_prompt_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_detail)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>][<pad>]Four-step Planning for Translating a Popular Science Article to Infographic Translation of Translation of a Scientific

In [ ]:
with model.disable_adapter():
  output_base_detail_b=get_completion(pred_prompt_detail_b, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
  print(output_base_detail_b)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
# Write and illustrate a children's book
children_book = {
    "Write Story": {
        "input": "Story idea",
        "output": "Written story",
        "steps": [
            "Brainstorm story ideas",
            "Create story outline",
            "Write first draft",
            "Revise and edit draft",
            "Finalize story"
        ],
        "failure_handling": [
            "If story idea is not engaging, brainstorm more ideas.",
            "If story outline is not clear, create a more detailed outline.",
            "If first draft is not satisfactory, revise and edit more thoroughly.",
            "If finalized story is not age-appropriate or developmentally suitable, revise and edit further."
        ]
    },
    "Illustrate": {
        "input": "Written story",
        "output": "Illustrated story",
        "steps": [
            "Sketch initial illustrations",
            "Refine and color illustrations",
            "Add text and labels to illustrations"


In [ ]:
output_base_detail_with=get_completion(pred_prompt_detail+additional_info_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
print(output_base_detail_with)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[<pad>][<pad>][<pad>]! Here's a detailed planning for translating a popular science article into another language and creating an infographic that summarizes it:

{'Translate the article': {'next_modes': ['Translate Text', 'Proofreading'], 'failure': ['Unable to find the article', 'Unable to access the article due to technical issues']},
'Translate Text': {'next_modes': ['Review Translation', 'Edit Translation'], 'failure': ['Inaccurate translation', 'Unclear translation']},
'Review Translation': {'next_modes': ['Finalize Translation', 'Revise Translation'], 'failure': ['Major errors in translation', 'Minor errors in translation']},
'Edit Translation': {'next_modes': ['Review Translation', 'Finalize Translation'], 'failure': ['Unable to edit translation']},
'Finalize Translation': {'next_modes': ['Create Infographic', 'Review Infographic'], 'failure': ['Unable to create infographic']},
'Create Infographic': {'next_modes': ['Review Infographic', 'Publish Infographic'], 'failure': ['Unab

In [ ]:
with model.disable_adapter():
  output_base_detail_with_b=get_completion(pred_prompt_detail_b+additional_info_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)
  print(output_base_detail_with_b)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#### Task: Write and illustrate a children's book, then create an audiobook version

#### Subtask: Write a children's book
{'detailed description of writing a children's book': {
'next_modes': ['Determine the story concept', 'Create a story outline', 'Write the first draft', 'Revise and edit the manuscript', 'Illustrate the manuscript'],
'failure': ['Unable to come up with a story concept', 'Unable to create a satisfactory story outline', 'Unable to write a coherent first draft', 'Unable to revise and edit the manuscript effectively', 'Unable to illustrate the manuscript']},

##### Determine the story concept
{'next_modes': ['Brainstorm ideas', 'Choose a concept'], 'failure': ['Unable to brainstorm ideas', 'Unable to choose a concept']},

##### Brainstorm ideas
{'next_modes': ['Research popular children's book genres', 'Consider personal experiences', 'Generate ideas from observations'], 'failure': ['Unable to find inspiration from research', 'Unable to generate ideas from personal exp

In [ ]:
adapter_id_blueprint='/content/drive/MyDrive/ckpts_mistral_7B_blueprint_planning_v2'
adapter_id_detail="/content/drive/MyDrive/ckpts_mistral_7B_detailed_planning_v2"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!unzip /content/drive/MyDrive/ckpts_mistral_7B_blueprint_planning_v2.zip -d .

Archive:  /content/drive/MyDrive/ckpts_mistral_7B_blueprint_planning_v2.zip
   creating: ./ckpts_mistral_7B_blueprint_planning_v2/
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/tokenizer.model  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/tokenizer_config.json  
 extracting: ./ckpts_mistral_7B_blueprint_planning_v2/added_tokens.json  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/special_tokens_map.json  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/adapter_model.safetensors  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/adapter_config.json  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/README.md  
  inflating: ./ckpts_mistral_7B_blueprint_planning_v2/tokenizer.json  


In [ ]:
!unzip /content/drive/MyDrive/ckpts_mistral_7B_detailed_planning_v2.zip -d .

Archive:  /content/drive/MyDrive/ckpts_mistral_7B_detailed_planning_v2.zip
   creating: ./ckpts_mistral_7B_detailed_planning_v2/
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/tokenizer.model  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/tokenizer_config.json  
 extracting: ./ckpts_mistral_7B_detailed_planning_v2/added_tokens.json  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/special_tokens_map.json  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/adapter_model.safetensors  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/adapter_config.json  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/README.md  
  inflating: ./ckpts_mistral_7B_detailed_planning_v2/tokenizer.json  


In [ ]:
model = PeftModel.from_pretrained(model,'/content/ckpts_mistral_7B_blueprint_planning_v2', adapter_name="blueprint")
_ = model.load_adapter('/content/ckpts_mistral_7B_detailed_planning_v2', adapter_name="detailed")

In [ ]:
%%time

adapters = ["blueprint", "detailed"]
weights = [1.0, 1.0]
adapter_name = "merge"
density = 0.2
sign_method="frequency"
combination_type = "ties"
if adapter_name in model.peft_config:
   model.delete_adapter(adapter_name)

model.add_weighted_adapter(
    adapters,
    weights,
    adapter_name,
    combination_type=combination_type,
    density=density,
    majority_sign_method=sign_method
    )

CPU times: user 2.54 s, sys: 7.9 ms, total: 2.55 s
Wall time: 1.11 s


In [ ]:
model.delete_adapter("blueprint")
model.delete_adapter("detailed")
model.set_adapter("merge")

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:626: UserWarning: Adapter blueprint was active which is now deleted. Setting active adapter to detailed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:626: UserWarning: Adapter detailed was active which is now deleted. Setting active adapter to merge.
  warnings.warn(


In [ ]:
prediction_blueprint_with=get_completion(pred_prompt_blueprint+additional_info_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_blueprint_with

"{'Translate article': {'next_modes': ['Create infographic'], 'failure': ['Retry with different translation software', 'Request human translation', 'Fallback to summary without visuals']},\n'Create infographic': {'next_modes': ['Analyzer'], 'failure': ['Retry with different design software', 'Fallback to text-only summary', 'Request human design assistance']}}</s>"

In [ ]:
failure_handling_options=['Retry with different translation software', 'Request human translation', 'Fallback to summary without visuals']
corresponding_weights=[8.4, 0., 1.5]
# Here we assign the smallest weight to the human intervention because we want error handling to be
# as autonomous as possible

failure_handling_options_weighted=generate_handle_permutations(failure_handling_options,corresponding_weights)
print(failure_handling_options_weighted)

[['Retry with different translation software', 'Fallback to summary without visuals', 'Request human translation'], ['Retry with different translation software', 'Request human translation', 'Fallback to summary without visuals'], ['Fallback to summary without visuals', 'Retry with different translation software', 'Request human translation'], ['Fallback to summary without visuals', 'Request human translation', 'Retry with different translation software'], ['Request human translation', 'Retry with different translation software', 'Fallback to summary without visuals'], ['Request human translation', 'Fallback to summary without visuals', 'Retry with different translation software']]


In [ ]:
prediction_blueprint_without=get_completion(pred_prompt_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_blueprint_without

'{\n"Translate article": {\n"next_modes": ["Create infographic"],\n"failure": ["Retry translation with different tools or settings", "Switch to a different article", "Request human translation"]\n},\n"Create infographic": {\n"next_modes": ["Review and finalize"],\n"failure": ["Retry with different design tools or templates", "Seek assistance from a graphic designer", "Fallback to a simpler format like a text summary"]\n},\n"Review and finalize": {\n"next_modes": [],\n"failure": None\n}\n}</s>'

In [ ]:
prediction_blueprint_with_b=get_completion(pred_prompt_blueprint_b+additional_info_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_blueprint_with_b

"{'Write story': {'next_modes': ['Illustrate story'], 'failure': ['Retry with different plot ideas', 'Switch to a different story', 'Request human intervention']},\n'Illustrate story': {'next_modes': ['Record audiobook'], 'failure': ['Retry with different illustrations', 'Switch to a different story', 'Request human intervention']},\n'Record audiobook': {'next_modes': ['Analyzer'], 'failure': ['Retry with different narration', 'Switch to a different story', 'Request human intervention']}}</s>"

In [ ]:
failure_handling_options=['Retry with different plot ideas', 'Switch to a different story', 'Request human intervention']
corresponding_weights=[5,3,0]

failure_handling_options_weighted=generate_handle_permutations(failure_handling_options,corresponding_weights)
print(failure_handling_options_weighted)

[['Retry with different plot ideas', 'Switch to a different story', 'Request human intervention'], ['Retry with different plot ideas', 'Request human intervention', 'Switch to a different story'], ['Switch to a different story', 'Retry with different plot ideas', 'Request human intervention'], ['Switch to a different story', 'Request human intervention', 'Retry with different plot ideas'], ['Request human intervention', 'Retry with different plot ideas', 'Switch to a different story'], ['Request human intervention', 'Switch to a different story', 'Retry with different plot ideas']]


In [ ]:
prediction_blueprint_without_b=get_completion(pred_prompt_blueprint_b, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_blueprint_without_b

'{\n"Write and illustrate a children\'s book": {\n"next_modes": ["Record narration for audiobook", "Edit and finalize illustrations"],\n"failure": ["Retry with different illustration styles", "Switch to a different project", "Request human intervention for creative direction"]\n},\n"Record narration for audiobook": {\n"next_modes": ["Edit and mix audio tracks"],\n"failure": ["Retry recording with improved equipment", "Use text-to-speech software as an alternative", "Seek professional voice actor assistance"]\n},\n"Edit and finalize illustrations": {\n"next_modes": ["Design cover art for the audiobook version"],\n"failure": ["Retry with different illustration techniques", "Outsource to a professional illustrator", "Consider simplifying the illustrations"]\n},\n"Edit and mix audio tracks": {\n"next_modes": ["Finalize audiobook production"],\n"failure": ["Retry editing with improved software", "Use pre-existing royalty-free audio", "Consider alternative narration styles"]\n},\n"Design cov

In [ ]:
test_data_90['Blueprint']

"{'Write story': {'next_modes': ['Illustrate story', 'Record audiobook'], 'failure': ['Retry', 'Fallback Content', 'Request Human Intervention']}, 'Illustrate story': {'next_modes': ['Record audiobook'], 'failure': ['Retry', 'Switch to Different Mode', 'Log and Analyze']}, 'Record audiobook': {'next_modes': ['Analyzer'], 'failure': ['Retry', 'Exit', 'Quality Assurance Check']}, 'Analyzer': {'next_modes': [], 'failure': None}}"

In [ ]:
prediction_detail_with=get_completion(pred_prompt_detail+additional_info_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_detail_with

"Here's a detailed planning for the given composite task:\n\n{'Translate the article': {'next_modes': ['Create an infographic'], 'failure': ['Retry with a different translation tool', 'Request human intervention for translation', 'Fallback to a summary of the article']},\n 'Create an infographic': {'next_modes': [], 'failure': ['Retry with a different design tool', 'Switch to a different format like a video or podcast', 'Fallback to a plain text summary']}}</s>"

In [ ]:
prediction_detail_without=get_completion(pred_prompt_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_detail_without

'{\n"Translate article": {\n"next_modes": ["Create infographic"],\n"failure": ["Retry translation with different tools or settings", "Switch to a different article", "Request human translation"]\n},\n"Create infographic": {\n"next_modes": ["Review and finalize"],\n"failure": ["Retry with different design elements or software", "Seek assistance from a graphic designer", "Fallback to a simpler format such as a text summary"]\n},\n"Review and finalize": {\n"next_modes": [],\n"failure": None\n}\n}</s>'

In [ ]:
pred_prompt_detail_b=format_prediction_detail_instruct_fn(test_data_90)

prediction_detail_b=get_completion(pred_prompt_detail_b+additional_info_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_detail_b

"### Example:\n{'Write and illustrate a children's book': {'next_modes': ['Record the audiobook version'], 'failure': ['Retry with different illustrations', 'Switch to a different project', 'Request human intervention']},\n'Record the audiobook version': {'next_modes': ['Analyzer'], 'failure': ['Retry with improved recording quality', 'Switch to a different narrator', 'Fallback to a pre-recorded version']},\n'Analyzer': {'next_modes': [], 'failure': None}}</s>"

In [ ]:
prediction_detail_without_b=get_completion(pred_prompt_detail_b, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prediction_detail_without_b

'{\n"Write and illustrate a children\'s book": {\n"next_modes": ["Record narration for audiobook", "Design and layout for print version"],\n"failure": ["Retry with different story ideas", "Switch to a different project", "Seek feedback from peers"]\n},\n"Record narration for audiobook": {\n"next_modes": ["Edit and finalize audiobook recording"],\n"failure": ["Retry with improved recording equipment", "Use text-to-speech software as alternative", "Consult a professional voice actor"]\n},\n"Edit and finalize audiobook recording": {\n"next_modes": ["Create cover art for audiobook"],\n"failure": ["Retry with different editing techniques", "Outsource editing to a professional", "Consider releasing an unedited version"]\n},\n"Create cover art for audiobook": {\n"next_modes": ["Publish both print and audiobook versions"],\n"failure": ["Retry with different design concepts", "Use stock art as temporary solution", "Delay publication until cover art is satisfactory"]\n},\n"Publish both print and

In [ ]:
test_data_90['Detailed']

'{"Develop a captivating plot for the children\'s book with a meaningful moral, and write the manuscript catering to a young audience": {\'next_modes\': [\'Create engaging and colorful illustrations that complement the story narrative\', \'Narrate and record the story with expressive voice modulation to captivate young listeners\'], \'failure\': ["Retry the story development with feedback from children\'s focus groups", \'Fallback Content by adapting well-known fairy tales with a unique twist\', \'Request Human Intervention for creative direction\']}, \'Create engaging and colorful illustrations that complement the story narrative, using a style that appeals to children\': {\'next_modes\': [\'Narrate and record the story with expressive voice modulation to captivate young listeners\'], \'failure\': [\'Retry illustration with different art styles or mediums based on feedback\', \'Switch to Different Mode by considering digital illustration techniques\', "Log and Analyze preferences in c

In [ ]:
from tqdm import tqdm

outputs_blueprint=[]
for i in tqdm(range(len(test_data_blueprint))):
  pred_prompt_blueprint_category=format_prediction_blueprint_instruct_fn(test_data_blueprint[i])

  output_blueprint=get_completion(pred_prompt_blueprint_category+additional_info_blueprint, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

  outputs_blueprint.append(output_blueprint)

print(outputs_blueprint)

  0%|          | 0/101 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/101 [00:12<21:14, 12.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 2/101 [00:37<32:45, 19.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  3%|▎         | 3/101 [00:54<30:22, 18.60s/it]The attention mask and the pad token id were not set. As a consequence, you may

["{'Develop VR tour of the solar system': {'next_modes': ['Write educational guide'], 'failure': ['Retry with different VR software', 'Switch to 2D animation', 'Request human intervention']},\n'Write educational guide': {'next_modes': ['Analyzer'], 'failure': ['Retry with different writing style', 'Use pre-existing content', 'Request human intervention']}}</s>", "{'Scripting': {'next_modes': ['Shooting product shots'], 'failure': ['Retry with different script ideas', 'Switch to a different scripting approach', 'Fallback to stock scripts']},\n'Shooting product shots': {'next_modes': ['Composing background music'], 'failure': ['Retry with different shooting angles', 'Switch to a different shooting location', 'Fallback to stock images']},\n'Composing background music': {'next_modes': ['Editing'], 'failure': ['Retry with different music styles', 'Switch to a different music source', 'Fallback to silence']},\n'Editing': {'next_modes': ['Analyzer'], 'failure': ['Retry with different editing 

In [ ]:
from tqdm import tqdm

outputs_detail=[]
for i in tqdm(range(len(test_data_detail))):
  pred_prompt_detail_category=format_prediction_detail_instruct_fn(test_data_detail[i])

  output_detail=get_completion(pred_prompt_detail_category+additional_info_detail, model, tokenizer, stop_keywords, KeywordsStoppingCriteria)

  outputs_detail.append(output_detail)

print(outputs_detail)

  0%|          | 0/101 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/101 [00:51<1:26:02, 51.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 2/101 [01:23<1:05:50, 39.91s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  3%|▎         | 3/101 [02:05<1:06:56, 40.99s/it]The attention mask and the pad token id were not set. As a consequence, y

["Here's a possible planning for the given composite task:\n\n{'Translate article into Language-1': {'next_modes': ['Translate article into Language-2', 'Translate article into Language-3'],\n'failure': ['Retry translation with a different translator', 'Switch to a different article', 'Log and analyze the issue']},\n'Translate article into Language-2': {'next_modes': ['Translate article into Language-3'], 'failure': ['Retry translation with a different translator', 'Switch to a different article', 'Log and analyze the issue']},\n'Translate article into Language-3': {'next_modes': ['Narrate article in Language-1', 'Narrate article in Language-2', 'Narrate article in Language-3'], 'failure': ['Retry translation with a different translator', 'Switch to a different article', 'Log and analyze the issue']},\n'Narrate article in Language-1': {'next_modes': ['Analyze narration quality'], 'failure': ['Retry narration', 'Switch to a different narrator', 'Log and analyze the issue']},\n'Narrate a

In [ ]:
test_data_with_blueprint_pred=test_data_blueprint.add_column("blueprint-pred-merge",outputs_blueprint)

Flattening the indices:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
df_test_with_blueprint_pred = test_data_with_blueprint_pred.to_pandas()

In [ ]:
df_test_with_blueprint_pred.head(5)

,Detailed,Blueprint,Instruction,prompt,response,input_ids,attention_mask,blueprint-pred-merge
0,{'Design and develop an immersive VR tour of t...,{'Develop VR tour': {'next_modes': ['Write edu...,Develop a virtual reality (VR) tour of the sol...,[INST]Below is an instruction that describes a...,### Response:{'Develop VR tour': {'next_modes'...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",{'Develop VR tour of the solar system': {'next...
1,{'Write a compelling script that highlights th...,{'Develop script': {'next_modes': ['Shoot prod...,Produce a promotional video for a new tech gad...,[INST]Below is an instruction that describes a...,### Response:{'Develop script': {'next_modes':...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",{'Scripting': {'next_modes': ['Shooting produc...
2,{'Produce an engaging narrated animation expla...,{'Create narrated animation': {'next_modes': [...,Design an interactive educational module for c...,[INST]Below is an instruction that describes a...,### Response:{'Create narrated animation': {'n...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",{'Narrated animation creation': {'next_modes':...
3,"{'Translate the original novel into English, S...","{'Translate novel': {'next_modes': ['English',...",Translate a novel into three languages and cre...,[INST]Below is an instruction that describes a...,### Response:{'Translate novel': {'next_modes'...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",{'Translate novel to Language-1': {'next_modes...
4,{'Design and code a functional mobile app prot...,{'Develop mobile app prototype': {'next_modes'...,"Develop a mobile app prototype, create a user ...",[INST]Below is an instruction that describes a...,### Response:{'Develop mobile app prototype': ...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",{'Develop mobile app prototype': {'next_modes'...


In [ ]:
test_data_with_detail_pred=test_data_detail.add_column("detail-pred-merge",outputs_detail)

Flattening the indices:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
df_test_with_detail_pred = test_data_with_detail_pred.to_pandas()

In [ ]:
df_test_with_detail_pred.head(5)

,Detailed,Blueprint,Instruction,prompt,response,input_ids,attention_mask,detail-pred-merge
0,{'Identify a science article with broad intern...,{'Select article': {'next_modes': ['Translate ...,Translate a popular science article into three...,[INST]Below is an instruction that describes a...,### Response:{'Identify a science article with...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Here's a possible planning for the given compo...
1,"{""Develop a creative concept that aligns with ...",{'Conceptualize design': {'next_modes': ['Crea...,Design a digital billboard ad for a fashion br...,[INST]Below is an instruction that describes a...,"### Response:{""Develop a creative concept that...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Here's a detailed planning for designing a dig...
2,"{""Research local wildlife species, their habit...",{'Gather information on local wildlife': {'nex...,Create a video documentary about local wildlif...,[INST]Below is an instruction that describes a...,"### Response:{""Research local wildlife species...","[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Here's a detailed planning for creating a vide...
3,{'Film a comprehensive guide on baking a choco...,{'Create a tutorial video': {'next_modes': ['W...,Create a tutorial video on baking a chocolate ...,[INST]Below is an instruction that describes a...,### Response:{'Film a comprehensive guide on b...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Here's a sample for the given composite task:\...
4,{'Select and review astronomy topics that appe...,{'Identify key astronomy concepts': {'next_mod...,Design an educational video series on basic as...,[INST]Below is an instruction that describes a...,### Response:{'Select and review astronomy top...,"[1, 733, 16289, 28793, 20548, 336, 349, 396, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Here's a possible planning for designing an ed...


In [ ]:
import json
import numpy as np


class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
# Convert DataFrame to a dictionary for prettier JSON
data_blueprint_dict = df_test_with_blueprint_pred.to_dict(orient='records')

dumped_blueprint = json.dumps(data_blueprint_dict, cls=NumpyEncoder)

# Writing prettified JSON to a file
with open('data_test_with_blueprint_pred_merge.json', 'w') as f:
    json.dump(dumped_blueprint, f, indent=4)

In [ ]:
# Convert DataFrame to a dictionary for prettier JSON
data_detail_dict = df_test_with_detail_pred.to_dict(orient='records')

dumped_detail = json.dumps(data_detail_dict, cls=NumpyEncoder)

# Writing prettified JSON to a file
with open('data_test_with_detail_pred_merge.json', 'w') as f:
    json.dump(dumped_detail, f, indent=4)

In [ ]:
from google.colab import files

files.download('data_test_with_blueprint_pred_merge.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

files.download('data_test_with_detail_pred_merge.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.peft_config

{'merge': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=8, target_modules={'gate_proj', 'k_proj', 'down_proj', 'o_proj', 'up_proj', 'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)}

In [ ]:
model.save_pretrained(".")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
!zip -r merge.zip merge/

  adding: merge/ (stored 0%)
  adding: merge/adapter_config.json (deflated 54%)
  adding: merge/adapter_model.safetensors (deflated 28%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!cp merge.zip '/content/drive/MyDrive'

In [ ]:
# from huggingface_hub import create_repo

# new_model = MERGED_MODEL_NAME

# user = whoami(token=hf_token)['name']
# repo_id = f'{user}/{new_model}'

# # Set exist_ok to False if the repo has not been created yet
# url = create_repo(repo_id, token=hf_token, exist_ok=True)

# print(url)

In [ ]:
# # Uncomment if you haven't created the model card
# from pathlib import Path

# from huggingface_hub import ModelCard, ModelCardData

# card_data = ModelCardData(language='en', license='MIT', library_name='torch')

# # Define your jinja template
# template_text = """
# ---
# {{ card_data }}
# ---

# # Model Card for Hierarchical_Planning_Decomposition_Mistral-7B-Instruct-v0.2

# ## Model Details

# ### Model Description

# This model decomposes a multi-modal composite task to actionable subtasks with two layers of abstraction.

# Layer-1 Abstract Planning: Dedicated to learning the decomposition of multi-modal composite tasks into keywords, modes for each subtask, and their ordered sequence for task completion.
# Layer-2 Detailed Planning: Focused on distilling tasks into detailed actionable subtasks, outlining the necessary steps for completing the composite task in order.


# - **Developed by:** Arash Shahmansoori
# - **Language(s) (NLP):** en
# - **License:** mit
# - **Finetuned from model:** Mistral-7B-v0.1

# ### Model Sources

# - **Repository:** https://github.com/arash-shahmansoori/hierarchical_planning_decompose_net.git
# """.strip()

# # Write the template to a file
# Path('custom_template.md').write_text(template_text)

# # Define card metadata
# card_data = ModelCardData(language='en', license='mit', library_name='torch')

# # Create card from template, passing it any jinja template variables you want.
# # In our case, we'll pass author
# card = ModelCard.from_template(card_data, template_path='custom_template.md', author='Arash Shahmansoori')
# card.save('decompose_net_card.md')
# print(card)
# card.push_to_hub(repo_id, token=hf_token)

In [ ]:
# model.push_to_hub(repo_id, token=hf_token)
# tokenizer.push_to_hub(repo_id, token=hf_token)